In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [14]:
def get_images_and_labels():
    data = pd.read_csv('zip.train.csv')
    labels = data.V1.to_numpy()
    images = data.drop(columns=['Unnamed: 0', 'V1']).to_numpy()
    return images, labels

In [15]:
images, labels = get_images_and_labels()

In [16]:
def showImage(pixels):
    img_pixels = pixels.reshape((16,16))
    plt.imshow(img_pixels, cmap='gray')
    plt.show()

In [17]:
def euclidean_distance(a, b):
    return np.linalg.norm(a-b)

In [18]:
def get_initial_centroids(k, data):
    return data[np.random.choice(data.shape[0], k, replace=False)]

In [10]:
def get_item_cluster_label(centroids, x):
    k = len(centroids)
    distances = np.zeros(k)
    
    for i in range(k):
        centroid = centroids[i]
        distances[i] = euclidean_distance(centroid, x)
        
    return np.argmin(distances)

In [11]:
def get_cluster_labels(data, centroids):
    cluster_labels = [None] * data.shape[0]
    
    for index in range(len(data)):
        item = data[index]
        label = get_item_cluster_label(centroids, item)
        cluster_labels[index] = label
        
    return np.array(cluster_labels)

In [12]:
def get_centroids(data, labels):
    unique_clusters = np.unique(labels)
    centroids = []
    
    for cluster_num in unique_clusters:
        data_idxs = np.where(labels == cluster_num)
        cluster_data = data[data_idxs]
        centroids.append(np.mean(cluster_data, axis=0))
        
    return np.array(centroids)

In [304]:
def k_means(data, k):
    centroids = get_initial_centroids(k, data)
    oldCentroinds = None
    iterations = 0
    
    while iterations < 4:
        iterations += 1
        oldCentroinds = centroids
        labels = get_cluster_labels(data, centroids)
        centroids = get_centroids(data, labels)
        
        print(labels)
        
    return labels

In [305]:
k_means(images, 10)

[7 7 8 ... 3 5 0]
[7 7 8 ... 3 5 0]
[7 7 8 ... 3 5 0]
[7 7 8 ... 3 5 0]


array([7, 7, 8, ..., 3, 5, 0])